In [11]:
import requests
import json
import mysql.connector
import hashlib as hs

import dbconnect as dbc

In [12]:
try:
    mydb = mysql.connector.connect(
        host=dbc.host,
        port=dbc.port,
        user=dbc.user,
        password=dbc.password#.encode(),
    )
except mysql.connector.Error as err:
    print(err)

In [13]:
url = 'https://www.pse.pl/transmissionMapService'
resp = requests.get(url=url).json()
print(resp)

{'status': '0', 'timestamp': 1711648530376, 'data': {'przesyly': [{'wartosc': 594, 'rownolegly': False, 'wartosc_plan': 600, 'id': 'SE'}, {'wartosc': 1815, 'rownolegly': True, 'wartosc_plan': 1263, 'id': 'DE'}, {'wartosc': -542, 'rownolegly': True, 'wartosc_plan': 93, 'id': 'CZ'}, {'wartosc': -145, 'rownolegly': True, 'wartosc_plan': -198, 'id': 'SK'}, {'wartosc': -192, 'rownolegly': False, 'wartosc_plan': -255, 'id': 'UA'}, {'wartosc': 0, 'rownolegly': False, 'wartosc_plan': 0, 'id': 'LT'}], 'podsumowanie': {'wodne': 1349, 'wiatrowe': 4604, 'PV': 0, 'generacja': 21151, 'zapotrzebowanie': 22725, 'czestotliwosc': 49.978, 'inne': 0, 'cieplne': 15199}}}


In [14]:
respDump = json.dumps(resp)
data = json.loads(respDump)

status = data['status']
timestamp = data['timestamp']
unit = "MW"

gene = data['data']['podsumowanie']
heatPower = gene['cieplne']#like coal, gas, NOT heat from earth
water = gene['wodne']
wind = gene['wiatrowe']
solar = gene['PV']
other = gene['inne']

production = gene['generacja']
consumption = gene['zapotrzebowanie']
frequency = gene['czestotliwosc']

transfer = data['data']['przesyly']
SE = [transfer[0]['wartosc'], transfer[0]['rownolegly'], transfer[0]['wartosc_plan']]
DE = [transfer[1]['wartosc'], transfer[1]['rownolegly'], transfer[1]['wartosc_plan']]
CZ = [transfer[2]['wartosc'], transfer[2]['rownolegly'], transfer[2]['wartosc_plan']]
SK = [transfer[3]['wartosc'], transfer[3]['rownolegly'], transfer[3]['wartosc_plan']]
UA = [transfer[4]['wartosc'], transfer[4]['rownolegly'], transfer[4]['wartosc_plan']]
LT = [transfer[5]['wartosc'], transfer[5]['rownolegly'], transfer[5]['wartosc_plan']]

In [17]:
dbcursor = mydb.cursor()

sql = f'INSERT INTO PSEdata.data VALUES ({status},{timestamp},"{unit}",{heatPower},{water},{wind},{solar},{other},{production},{consumption},{frequency}, {SE[0]}, {SE[2]}, {SE[1]}, {DE[0]}, {DE[2]}, {DE[1]}, {CZ[0]}, {CZ[2]}, {CZ[1]}, {SK[0]}, {SK[2]}, {SK[1]}, {UA[0]}, {UA[2]}, {UA[1]}, {LT[0]}, {LT[2]}, {LT[1]}, 0);'
print(sql)
dbcursor.execute(sql)
mydb.commit()
print(dbcursor.rowcount, "record inserted.")

dbcursor.close()

INSERT INTO PSEdata.data VALUES (0,1711648530376,"MW",15199,1349,4604,0,0,21151,22725,49.978, 594, 600, False, 1815, 1263, True, -542, 93, True, -145, -198, True, -192, -255, False, 0, 0, False, 0);
1 record inserted.


True